# Example of ingesting US Equities data 

This example captures US Equities data using [Databento´s API](https://databento.com/docs/venues-and-datasets/equs-mini) and stores the data into QuestDB.

In order to execute this script, you need to provide a Databento API. If you don't have one, you can register and start a free trial. Please refer to [Databento pricing](https://databento.com/pricing) for more details.

The script will connect to the Databento API and subscribe to live data about US Equities trades for the symbols `AMZN,AAPL,MSFT,GOOG,NVDA,META,TSLA,NFLX,ORCL,QSG,BABA`. Depending on the time you are executing the script, it might be the case there is very low trading volume, in which case you can just comment the line with the symbols on the script to capture the data from the feed for all available symbols.

The way Databento API provides the data, it sends messages of different type within the same feed. For the purposes of this demo, a message will be either a Trade (TradeMsg) or a Symbol Mapping (SymbolMappingMsg). Each trading message comes with an `instrument_id` and we can use the mapping messages to look up the symbol name. The script keeps an `instrument` array in memory mapping `id` to `name`. 

In this case, we are using the Python QuestDB client library for convenience, but QuestDB offers similar native clients for C/C++, Java, .Net, Rust, Go, and JavaScript.

Each trading message follows [this schema](https://databento.com/docs/schemas-and-data-formats/trades?historical=python&live=python&reference=python). The script below processes each message and it stores it
on QuestDB, in a table named `trades`, with the schema below. If table does not exist, it will be automatically created on the first write.

```sql
CREATE TABLE 'trades' ( 
	symbol SYMBOL CAPACITY 256 CACHE,
	side SYMBOL CAPACITY 256 CACHE,
	price DOUBLE,
	amount DOUBLE,
	timestamp TIMESTAMP
) timestamp(timestamp) PARTITION BY DAY WAL;
```

To see the live data on your database, you can open a new tab on your browser and navigate to `http://localhost:9000`. You can then execute a simple query like `SELECT * FROM trades -10;` to see the latest 10 trades. Or you could execute a sligthly more sophisticated query like `select timestamp, symbol, side, sum(price * amount) from trades sample by 1m ` to get the totals for each symbol at 1 minute intervals.

If you want some more realistic queries, please open in a new tab the [Examples-of-market-data-queries notebook](/notebooks/Examples-of-market-data-queries.ipynb), where you will find some queries adapted from the demo machine that should return results for your dataset.

If you want to see your live data on a real-time dashboard, please navigate in a new tab to [the demo dashboard](http://localhost:3000/d/trades-crypto-currency/trades-crypto-currency?orgId=1&refresh=250ms) powered by Grafana. The user is `admin` and password `quest`




In [ ]:
import databento as db
from datetime import datetime
from questdb.ingress import Sender, TimestampNanos
import numpy as np
import os

DATABENTO_API_KEY = os.getenv('DATABENTO_API_KEY', '<YOUR_API_KEY>') 

# Opening the connection to the QuestDB instance 
questdb_conf = "http::addr=host.docker.internal:9000;auto_flush=on;auto_flush_rows=100;auto_flush_interval=1000;"
sender = Sender.from_conf(questdb_conf)
sender.establish()
# First we build up the static data dictionary
instruments = {}

# Enable some basic logging
db.enable_logging("WARN")

# Create a live client and connect
live_client = db.Live(
    key=DATABENTO_API_KEY,
    reconnect_policy="reconnect"
)

# Subscribe to the ohlcv-1s schema for AMZN
live_client.subscribe(
    dataset="EQUS.MINI",
    schema="trades",
    stype_in="raw_symbol",
    #you can comment the next line if you want to get all the stocks
    #this is specially interesting if executing this outside of Nasdaq market hours
    symbols="AMZN,AAPL,MSFT,GOOG,NVDA,META,TSLA,NFLX,ORCL,QSG,BABA",
)


# Create a callback to handle DBN records
def record_callback(record: db.DBNRecord) -> None:    
  if isinstance(record, db.SymbolMappingMsg):     
    instruments.update({record.hd.instrument_id : record.stype_out_symbol})
      
  if isinstance(record, db.TradeMsg):  
    instrument = instruments[record.instrument_id]
    size = record.size
    if record.action == 'A':
        side='ask'
    else:
        side='buy'
        
    # databento provides the price where every 1 unit corresponds to 1e-9, i.e. 1/1,000,000,000 or 0.000000001.    
    price = record.price * 0.000000001   

    # This is where we send data to the QuestDB instance  
    sender.row(
      'trades',
    symbols={'symbol': instrument},
    columns={'amount': size,
             'price': price,
             'side': side
            },
             at=TimestampNanos(record.ts_event)
    )

   

# Create a callback to handle reconnections
def reconnect_callback(start, end) -> None:
    sender.flush()
    sender.close()
    sender.establish()
    print(f"reconnection gap from {start} to {end}")


# Create a callback to handle exceptions from `user_callback`
def error_handler(exception: Exception) -> None:
    print(f"an error occurred {exception}")

# The callback we register here will be invoked for each record
live_client.add_callback(
    record_callback=record_callback,
    exception_callback=error_handler
)

live_client.add_reconnect_callback(
    reconnect_callback=reconnect_callback,
    exception_callback=error_handler,  # optional error handler
)

# Start the live client to begin data streaming
live_client.start()

print("The process will run in the background until the notebook is stopped.")

# Run the stream for 15 seconds before closing
live_client.block_for_close(timeout=None)

The process will run in the background until the notebook is stopped.
